In [1]:
###
# Classify all data
# Sp
###

In [2]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import rasterio as rio
import pathlib

In [3]:
model = tf.keras.models.load_model('/Users/brianlee/Desktop/new_bats/data/doppler/20220502_new_model/2018/20180620/20230426_ann_tf_3hidden_50E_norm/model/')


In [4]:
# Try original dataset
raw_df = pd.read_csv('/Users/brianlee/Desktop/new_bats/data/training/sources/yolo_csv/yolo_training.csv')
# Although data are mostly pre-processed, drop nan and convert to binary for classifier
cleaned_df = raw_df.dropna()
# Reclassify yolo training data to binary
# data = data.replace({'class': {10: 0, 11:0, 12:0, 13:0}})
cleaned_df = cleaned_df.replace({'training_class': {10: 0, 11:0, 12:0, 13:0}})
cleaned_df = cleaned_df.drop('date', axis=1)
# Use a utility from sklearn to split and shuffle your dataset.
train_df, test_df = train_test_split(cleaned_df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)
# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('training_class'))
# bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('training_class'))
test_labels = np.array(test_df.pop('training_class'))

variables = ['cor', 'pha', 'dif', 'ref', 'spw', 'vel']

train_features = np.array(train_df[variables])
val_features = np.array(val_df[variables])
test_features = np.array(test_df[variables])
# normalize data instead of standardize
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(train_features, batch_size=32)

2023-04-28 06:24:10.636816: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [5]:
def classify_image(new_image, file, model, normalizer, classdir):
    # Read the rasters as array
    
    new_image = os.path.abspath(new_image)
    
    features1=rio.open(new_image)


    # # use reshape here.
    # cor_features = ndimage.median_filter(np.array(features1.read(1)), 5).flatten()
    # pha_features = ndimage.median_filter(np.array(features1.read(2)), 5).flatten()
    # dif_features = ndimage.median_filter(np.array(features1.read(3)), 5).flatten()
    # ref_features = ndimage.median_filter(np.array(features1.read(4)), 5).flatten()
    # spw_features = ndimage.median_filter(np.array(features1.read(5)), 5).flatten()
    # vel_features = ndimage.median_filter(np.array(features1.read(6)), 5).flatten()

    cor_features = features1.read(1).flatten()
    pha_features = features1.read(2).flatten()
    dif_features = features1.read(3).flatten()
    ref_features = features1.read(4).flatten()
    spw_features = features1.read(5).flatten()
    vel_features = features1.read(6).flatten()



    # ds3, featuresimage2_scratch = raster.read(new_image, bands='all')

    stacked_features = np.column_stack((cor_features, pha_features, dif_features, ref_features, spw_features, vel_features))
    # df = pd.DataFrame(stacked_features, columns=['b1', 'b2', 'b3', 'b4', 'b5', 'b6'])
    df = pd.DataFrame(stacked_features, columns=['cor', 'pha', 'dif', 'ref', 'spw', 'vel'])

    variables = ['cor', 'pha', 'dif', 'ref', 'spw', 'vel']
    # df['vel'] = df['vel'].abs()
    # variables = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6']
    numeric_features = df[variables]
    numeric_features_array = numeric_features.to_numpy()

    # numeric_features_array = scaler.fit_transform(numeric_features_array)


    # normalize features
    # normalizer = tf.keras.layers.experimental.preprocessing.Normalization(axis=-1)
    # normalizer.adapt(numeric_features_array)  
    numeric_features_array_norm = normalizer(numeric_features_array)

    predicted = model.predict(numeric_features_array_norm, batch_size=1000, workers=10, use_multiprocessing=True)
    # predicted = predicted[:,1]

    #Export raster
    prediction = np.reshape(predicted, (features1.height, features1.width))

    # Convert values less than 0.7 prob to 0
    prediction[np.abs(prediction) < 0.9] = 0

    outFile = str(classdir) + '/' + 'classified_' + file

    with rio.Env():

        # Write an array as a raster band to a new 8-bit file. For
        # the new file's profile, we start with the profile of the source
        profile = features1.profile

        # And then change the band count to 1, set the
        # dtype to uint8, and specify LZW compression.
        profile.update(
            dtype=rio.float32,
            count=1,
            compress='lzw')

        with rio.open(outFile, 'w', **profile) as dst:
            dst.write(prediction.astype(rio.float32), 1)


In [6]:
os.chdir('/Volumes/backupdata/doppler/data/2018')

In [7]:
rootdir = os.getcwd()

In [8]:
classdir = pathlib.Path('/Volumes/backupdata/doppler/data/classified')

In [12]:
for root, dirs, files in os.walk(rootdir):
    for file in files:
        if file.endswith('.tif'):
            # print(file)
            classified_file = 'classified_' + file 
            image_file = (os.path.join(classdir, classified_file))
            # print(image_file)
            if os.path.exists(image_file) == False:
                image_file2 = (os.path.join(root, file))
                # print('image does not exist')
                classify_image(image_file2, file, model, normalizer, classdir)

/Volumes/backupdata/doppler/data/classified/classified_20180909_0218.tif
/Volumes/backupdata/doppler/data/classified/classified_20180909_0634.tif
/Volumes/backupdata/doppler/data/classified/classified_20180909_0620.tif
/Volumes/backupdata/doppler/data/classified/classified_20180909_0812.tif
/Volumes/backupdata/doppler/data/classified/classified_20180909_0225.tif
/Volumes/backupdata/doppler/data/classified/classified_20180909_0958.tif
/Volumes/backupdata/doppler/data/classified/classified_20180909_0805.tif
/Volumes/backupdata/doppler/data/classified/classified_20180909_0232.tif
/Volumes/backupdata/doppler/data/classified/classified_20180909_0544.tif
/Volumes/backupdata/doppler/data/classified/classified_20180909_0418.tif
/Volumes/backupdata/doppler/data/classified/classified_20180909_0342.tif
/Volumes/backupdata/doppler/data/classified/classified_20180909_1048.tif
/Volumes/backupdata/doppler/data/classified/classified_20180909_0356.tif
/Volumes/backupdata/doppler/data/classified/classif

In [15]:
classdir

PosixPath('/Volumes/backupdata/doppler/data/classified')

In [ ]:

for root, dirs, files in os.walk(rootdir):
    for file in files:
        if file.endswith('.tif'):
            image_file = (os.path.join(root, file))
            # try:
                
            new_image = os.path.abspath(image_file)
            print(new_image)

            features1=rio.open(new_image)


            # # use reshape here.
            # cor_features = ndimage.median_filter(np.array(features1.read(1)), 5).flatten()
            # pha_features = ndimage.median_filter(np.array(features1.read(2)), 5).flatten()
            # dif_features = ndimage.median_filter(np.array(features1.read(3)), 5).flatten()
            # ref_features = ndimage.median_filter(np.array(features1.read(4)), 5).flatten()
            # spw_features = ndimage.median_filter(np.array(features1.read(5)), 5).flatten()
            # vel_features = ndimage.median_filter(np.array(features1.read(6)), 5).flatten()

            cor_features = features1.read(1).flatten()
            pha_features = features1.read(2).flatten()
            dif_features = features1.read(3).flatten()
            ref_features = features1.read(4).flatten()
            spw_features = features1.read(5).flatten()
            vel_features = features1.read(6).flatten()



            # ds3, featuresimage2_scratch = raster.read(new_image, bands='all')

            stacked_features = np.column_stack((cor_features, pha_features, dif_features, ref_features, spw_features, vel_features))
            # df = pd.DataFrame(stacked_features, columns=['b1', 'b2', 'b3', 'b4', 'b5', 'b6'])
            df = pd.DataFrame(stacked_features, columns=['cor', 'pha', 'dif', 'ref', 'spw', 'vel'])

            variables = ['cor', 'pha', 'dif', 'ref', 'spw', 'vel']
            # df['vel'] = df['vel'].abs()
            # variables = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6']
            numeric_features = df[variables]
            numeric_features_array = numeric_features.to_numpy()

            # numeric_features_array = scaler.fit_transform(numeric_features_array)


            # normalize features
            # normalizer = tf.keras.layers.experimental.preprocessing.Normalization(axis=-1)
            # normalizer.adapt(numeric_features_array)  
            numeric_features_array_norm = normalizer(numeric_features_array)

            predicted = model.predict(numeric_features_array_norm)
            # predicted = predicted[:,1]

            #Export raster
            prediction = np.reshape(predicted, (features1.height, features1.width))

            # Convert values less than 0.7 prob to 0
            prediction[np.abs(prediction) < 0.9] = 0

            outFile = str(classdir) + '/' + 'classified_' + file
            print(outFile)

            with rio.Env():

                # Write an array as a raster band to a new 8-bit file. For
                # the new file's profile, we start with the profile of the source
                profile = features1.profile

                # And then change the band count to 1, set the
                # dtype to uint8, and specify LZW compression.
                profile.update(
                    dtype=rio.float32,
                    count=1,
                    compress='lzw')

                with rio.open(outFile, 'w', **profile) as dst:
                    dst.write(prediction.astype(rio.float32), 1)
            
            # except:
            #     print(file + ' did not work')
            #     pass



/Volumes/backupdata/doppler/data/2018/20180908/2_scan_agg/20180909_0218.tif
122735/122735 [==============================] - 41s 334us/step
/Volumes/backupdata/doppler/data/classified/classified_20180909_0218.tif
/Volumes/backupdata/doppler/data/2018/20180908/2_scan_agg/20180909_0634.tif
122735/122735 [==============================] - 41s 337us/step
/Volumes/backupdata/doppler/data/classified/classified_20180909_0634.tif
/Volumes/backupdata/doppler/data/2018/20180908/2_scan_agg/20180909_0620.tif
122735/122735 [==============================] - 41s 333us/step
/Volumes/backupdata/doppler/data/classified/classified_20180909_0620.tif
/Volumes/backupdata/doppler/data/2018/20180908/2_scan_agg/20180909_0812.tif
122735/122735 [==============================] - 41s 331us/step
/Volumes/backupdata/doppler/data/classified/classified_20180909_0812.tif
/Volumes/backupdata/doppler/data/2018/20180908/2_scan_agg/20180909_0225.tif
122735/122735 [==============================] - 41s 331us/step
/Volumes